In [1]:
from __future__ import print_function
from __future__ import division

import pandas
import numpy
import scipy
import matplotlib
import seaborn
import sklearn
import sys

import itertools

from matplotlib import pyplot

# Classification

In [2]:
features = pandas.read_csv('dev/features.csv')

/usr/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253

In [3]:
from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression

In [7]:
X = features.drop('spam', axis = 1)
y = features.spam

%xdel features

In [21]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, train_size = .5, random_state = 0)

## Logistic Regression

In [14]:
lr = LogisticRegression(n_jobs = -1)
lr.fit(X_train, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
lr.

## Decision Tree

In [5]:
from sklearn.tree import DecisionTreeClassifier

In [48]:
dt = DecisionTreeClassifier(criterion = 'entropy', random_state = 0, splitter = 'random', max_features = 'sqrt', class_weight = 'balanced')
dt.fit(X_train.fillna(0), y_train.fillna(0))

DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='random')

In [49]:
dt.score(X_test.fillna(0), y_test.fillna(0))

0.99254320987654321

In [70]:
options = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_features': ['sqrt', 'log2']
}

options = [{k: v for k, v in zip(options.keys(), x)} for x in itertools.product(*options.values())]

In [84]:
q = []

for p in options:
    dt = DecisionTreeClassifier(random_state = 0, **p)
    dt.fit(X_train.fillna(0), y_train.fillna(0))
    q.append((dt.score(X_test.fillna(0), y_test.fillna(0)), p))

sorted(q, reverse = True)

[(0.99506172839506168,
  {'criterion': 'gini', 'max_features': None, 'splitter': 'random'}),
 (0.99486419753086419,
  {'criterion': 'entropy', 'max_features': None, 'splitter': 'random'}),
 (0.9946666666666667,
  {'criterion': 'gini', 'max_features': 'sqrt', 'splitter': 'random'}),
 (0.99451851851851847,
  {'criterion': 'entropy', 'max_features': None, 'splitter': 'best'}),
 (0.99377777777777776,
  {'criterion': 'gini', 'max_features': None, 'splitter': 'best'}),
 (0.99259259259259258,
  {'criterion': 'gini', 'max_features': 'log2', 'splitter': 'best'}),
 (0.99239506172839509,
  {'criterion': 'entropy', 'max_features': 'sqrt', 'splitter': 'random'}),
 (0.99209876543209874,
  {'criterion': 'entropy', 'max_features': 'log2', 'splitter': 'best'}),
 (0.99111111111111116,
  {'criterion': 'entropy', 'max_features': 'sqrt', 'splitter': 'best'}),
 (0.99037037037037035,
  {'criterion': 'gini', 'max_features': 'sqrt', 'splitter': 'best'}),
 (0.98997530864197536,
  {'criterion': 'entropy', 'max_f

In [29]:
dt = DecisionTreeClassifier(random_state = 0, splitter = 'random', max_features = 'sqrt', max_depth = 6)
dt.fit(X_train.fillna(.5), y_train.fillna(.5))
dt.score(X_test.fillna(.5), y_test.fillna(.5))

0.9996049382716049

In [30]:
sklearn.tree.export_graphviz(dt.tree_, out_file = 'sqrt.dot', class_names = ['ham', 'spam'], feature_names = X.columns, filled = True, proportion = True)
!dot -Tpng sqrt.dot > sqrt.png

# Newer and better

In [2]:
columns = pandas.read_csv('dev/features_all.csv', nrows = 0).columns
duplicated = numpy.ones_like(columns, dtype = bool)

In [3]:
sys.setrecursionlimit(10000)
for e, c in enumerate(pandas.read_csv('dev/features_all.csv', chunksize = 10000)):
    print('Chunk {}'.format(e), file = sys.stderr)
    duplicated &= c.T.duplicated().T

duplicated.sum()

Chunk 0
Chunk 1
Chunk 2
Chunk 3
Chunk 4
Chunk 5
Chunk 6
Chunk 7
Chunk 8


1658

In [12]:
(columns == 'spam').argmax()

5070

In [13]:
duplicated[5070]

False

In [16]:
dedup_columns = columns[~duplicated]

for e, c in enumerate(pandas.read_csv('dev/features_all.csv', chunksize = 10000)):
    print('Chunk {}'.format(e), file = sys.stderr)
    c[dedup_columns].to_csv('dev/features_dedup.csv', header = e == 0, index = False, mode = 'w' if e == 0 else 'a')

Chunk 0
Chunk 1
Chunk 2
Chunk 3
Chunk 4
Chunk 5
Chunk 6
Chunk 7
Chunk 8


In [15]:
duplicated.shape

(5071,)

# OneHotEncoding

## Getting allowable columns

In [47]:
head = pandas.read_csv('novi/train_shuf.csv', nrows = 25000, dtype = object, index_col = 'num').drop('spam', axis = 1)

In [71]:
cols = head[head.columns[head.apply(lambda x: (x.fillna('').str.strip().str.len() > 0).sum() >= 25)]].apply(lambda x: x.str.strip().nunique() < 10 and x.str.strip().nunique() > 1)
dict_columns = cols[cols].index

## OneHotEncoder on this subset

In [74]:
from sklearn.feature_extraction import DictVectorizer

In [75]:
table = pandas.read_csv('novi/train.csv', usecols = dict_columns | ['num'], dtype = object, index_col = 'num')

In [76]:
dv = DictVectorizer(sparse = False)
categorized = pandas.DataFrame(dv.fit_transform(table.to_dict(orient = 'records')), columns = map(str.strip, dv.get_feature_names()), index = table.index)
valid = categorized.columns[~categorized.isnull().any()]

In [77]:
categorized[valid].astype(int).to_csv('novi/feature_header_categories.csv', header = True, index = True)

In [54]:
tab

['Content-transfer-encoding',
 'Content-transfer-encoding= 7BIT',
 'Content-transfer-encoding= 7Bit',
 'Content-transfer-encoding= 7bit',
 'Content-transfer-encoding= 8BIT',
 'Content-transfer-encoding= 8bit',
 'Content-transfer-encoding= quoted-printable',
 'Importance',
 'Importance= ',
 'Importance= High',
 'Importance= Medium',
 'Importance= Normal',
 'Importance= Normal ',
 'Importance= high',
 'Importance= low',
 'Importance= normal',
 'Mailing-List',
 'Mailing-List= contact customers-help@list.frontierairlines.com; run by ezmlm',
 'Mailing-List= contact dm-kz1-help@mail.dmx0.com; run by ezmlm',
 'Mailing-List= contact entertainment3-help@ronald.joke-land.com; run by ezmlm',
 'Mailing-List= contact foxsportsflash_h-help@mailings.foxsports.com; run by ezmlm',
 'Mailing-List= contact quickflicks-help@free-irewards.com; run by ezmlm',
 'Mailing-List= list ASME-OMAE-Houston@yahoogroups.com; contact ASME-OMAE-Houston-owner@yahoogroups.com',
 'Mailing-List= list EnerfaxGOLD@yahoogroups

# Whitespace

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

